In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocesado de los datos.

Nuestro objetivo es que el modelo a realizar detecte donde estan los ojos cuando le demos una foto. Para ello lo primero que necesitamos son datos. La Universidad de Illinois nos ofrece el dataset al que llaman <a href="http://www.ifp.illinois.edu/~vuongle2/helen/">Helen Dataset</a>.. Este contiene 2000 imagenes con anotaciones de las principales areas faciales, en la que se incluyen los ojos.

Debemos "limpiar" el dataset para solo quedarnos con los datos que corresponden a los ojos y transforman los datos al formato que queremos.

# Cargar las imagenes

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from skimage.io import imread
import json
import cv2
helen_dir = '/content/drive/MyDrive/EyePosition/dataset/'
def read_annot_file(in_path):
    with open(in_path, 'r') as f:
        file_id = f.readline().strip()
    out_df = pd.read_csv(in_path, skiprows=1, header=None)
    out_df.columns = ['x', 'y']
    out_df['file_id'] = file_id
    return out_df

annot_df = pd.concat([
    read_annot_file(c_path).reset_index()
    for c_path in 
    glob(os.path.join(helen_dir, 'annotation', '*', '*'))
])
annot_df.sample(3)

,index,x,y,file_id
101,101,880.28,1014.39,2397601939_1
141,141,1199.25,903.11,2685003351_1
166,166,1280.36,466.77,2198286445_1


# Cargar las anotaciones

In [ ]:
all_image_dict = {os.path.splitext(f)[0]: os.path.join(p, f) 
              for p, _, files in os.walk(helen_dir) 
              for f in files if f.upper().endswith('JPG')}

annot_df['path'] = annot_df['file_id'].map(all_image_dict.get)
annot_df.dropna(inplace=True)

Se observa que los datos usan el formato FUT estandar que consiste en lo siquiente:
<ul>
  <li>Linea de la cara: 41 puntos</li>
  <li>Linea de la nariz: 17 puntos</li>
  <li>Linea de los ojos: 20 puntos (por cada ojo)</li>
  <li>Linea de las cejas: 20 puntos (por cada ojo)</li>
  <li>Linea de la boca: 28 puntos (por cada labio)</li>
</ul>

Teniendo en cuenta esto es dificil poner una etiqueta a cada punto dependendiendo de a que parte de la cara corresponde. Es lo que el codigo de abajo hace:

In [ ]:
fig, m_axs = plt.subplots(3, 3, figsize = (30, 30))
marker_id = ['face', 'nose', 'mouth_inner', 'mouth_outer', 'r_eye', 'l_eye', 'r_eyebrow', 'l_eyebrow']
marker_split = np.cumsum([0, 41, 17, 28, 28, 20, 20, 20, 20])
marker_dict = {marker_id[i]: (marker_split[i], marker_split[i+1]) for i in range(len(marker_split)-1)}
for c_ax, (c_path, c_rows) in zip(m_axs.flatten(), annot_df.groupby('path')):
    img = imread(c_path)
    c_ax.imshow(img)
    for label, (start, end) in marker_dict.items():
        n_rows = c_rows.query(f'index>={start} and index<={end}')
        c_ax.plot(n_rows['x'], n_rows['y'], '.', label=label)
    c_ax.legend()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:

point_map = {i: [k for k, (n,x) in marker_dict.items() if n<=i<x] 
 for i in range(annot_df['index'].max()+1) }
annot_df['body_part'] = annot_df['index'].map(lambda x: point_map.get(x)[0])
annot_df['body_part'].value_counts()

face           82000
mouth_inner    56000
mouth_outer    56000
r_eye          40000
l_eye          40000
r_eyebrow      40000
l_eyebrow      40000
nose           34000
Name: body_part, dtype: int64

Una vez hecho esto gracias a las facilidades que nos da la libreria pandas se pueden filtrar los puntos para asi solo quedarnos con los que necesitamos, en nuestro caso los ojos.

In [ ]:
filt_df = annot_df[annot_df['body_part'].isin(['l_eye', 'r_eye'])]
fig, m_axs = plt.subplots(3, 3, figsize = (30, 30))
for c_ax, (c_path, c_rows) in zip(m_axs.flatten(), filt_df.groupby('path')):
    img = imread(c_path)
    c_ax.imshow(img)
    for label, (start, end) in marker_dict.items():
        n_rows = c_rows.query(f'index>={start} and index<={end}')
        c_ax.plot(n_rows['x'], n_rows['y'], '.')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
file_id = []
for file in filt_df['file_id']:
  if file not in file_id:
      file_id.append(file)
len(file_id)

df = filt_df[ filt_df.file_id == file_id[0]]
df = df.reset_index()
df['body_part'].value_counts()
df.index

RangeIndex(start=0, stop=40, step=1)

Por ultimo, vamos a transformar los datos en un formato en el que me siento mas comodo, ya que he trabajado previamente con el. Se trata de guardar los puntos en un archivo json de la siguiente forma:

{

  "image": "filename.jpg",

  "class": [1,1],
  
  "keypoints": (Puntos de los ojos)

}

Tras esto se guardan todos los puntos en su correspondiente archivo json y ya estarian todos los datos preparados para entrenar el modelo

In [ ]:
keypoints = []
for file in file_id:

  df = filt_df[filt_df.file_id == file]
  df = df.drop('index',axis=1)
  df = df.reset_index()
  img = cv2.imread('/content/drive/MyDrive/img/{}.jpg'.format(file))
  
  h,w,c = img.shape
  #for i in df_left.index:
    
    #   row = df_left.iloc[i]
    #  keypoints.append(row['x'])
    #   keypoints.append(row['y'])
  for i in range(0,len(df.index)):
    
      row = df.iloc[i]
      keypoints.append(row['x']/w)
      keypoints.append(row['y']/h)

  if len(keypoints) > 0:
    js = { "image": "{}.jpg".format(file), "class" : [1,1],"keypoints": keypoints}
    with open('/content/drive/MyDrive/annotation/{}.json'.format(file), 'w') as f:
      json.dump(js, f, indent=2)
    keypoints.clear()




In [ ]:
import cv2
import json
import time
from google.colab.patches import cv2_imshow
with open(r'/content/drive/MyDrive/EyePosition/dataset_format/train_1/annotation/100040721_1.json',) as f:
  file = json.load(f)
  keypoints = file['keypoints']

img = cv2.imread('/content/drive/MyDrive/EyePosition/dataset_format/train_1/train_1/100040721_1.jpg')
h,w,c = img.shape
print(len(keypoints))
for i in range(0,len(keypoints)-1,2):

    cv2.circle(img, [int(keypoints[i])*w, int(keypoints[i+1])*h], 2, (255,0,0), -1)

cv2_imshow(img)



In [ ]:
!zip -r /content/annotation.zip /content/drive/MyDrive/annotation
